In [10]:
import json
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import snntorch
import nirtorch
from snntorch import import_nirtorch
import matplotlib.pyplot as plt


In [4]:
### TEST DATA
test_data_path = "data/ds_test.pt"
ds_test = torch.load(test_data_path)

letter_written = ['Space', 'A', 'E', 'I', 'O', 'U', 'Y']
device = "cpu"
### LOSS FUNCTION
loss_fn = torch.nn.functional.cross_entropy
### OPTIMAL HYPERPARAMETERS
parameters_path = "data/parameters_ref_zero.json"

with open(parameters_path) as f:
   parameters = json.load(f)

import nir
# nir_graph = nir.read("braille_retrained_zero.nir")
nir_graph = nir.read("braille_noDelay_bias_zero.nir")
# nir_graph


In [5]:
# [type(x) for x in nir_graph.nodes.values()]


In [6]:
nir_graph = nir.read("braille_noDelay_bias_zero.nir")
net = import_nirtorch.from_nir(nir_graph)
net


replace rnn subgraph with nirgraph


GraphExecutor(
  (fc1): Linear(in_features=12, out_features=38, bias=True)
  (fc2): Linear(in_features=38, out_features=7, bias=True)
  (input): Identity()
  (lif2): Synaptic()
  (output): Identity()
  (lif1): RSynaptic(
    (recurrent): Linear(in_features=38, out_features=38, bias=True)
  )
)

In [7]:
nir_graph.nodes.keys()


dict_keys(['fc1', 'fc2', 'input', 'lif2', 'output', 'lif1'])

# Save activity of first layer

In [9]:
def record_layer1(graph):
    net = import_nirtorch.from_nir(nir.read(graph))
    lin_out = net.fc1(ds_test[0][0])
    lin_out.shape
    lif1_out = []
    state = None
    for i, t in enumerate(lin_out):
        z, state = net.lif1(t, state)
        lif1_out.append(z)
    return torch.stack(lif1_out).detach()
zero_out = record_layer1("braille_noDelay_bias_zero.nir")
zero_out.shape


replace rnn subgraph with nirgraph


AttributeError: 'RSynaptic' object has no attribute 'mem'

In [7]:
np.save("snntorch_activity_zero.npy", zero_out.numpy())


In [19]:
subtract_out = record_layer1("braille_subtract.nir", dt=1e-4, reset_method=norse.torch.functional.reset_subtract)
subtract_out.shape
np.save("snntorch_activity_subtract.npy", zero_out.numpy())


TypeError: record_layer1() got an unexpected keyword argument 'dt'

In [20]:
def apply(net, data):
    state = None
    hid_rec = []
    out = []
    for i, t in enumerate(data):
        z, state = net(t, state)
        out.append(z)
        hid_rec.append(state)
    spk_out = torch.stack(out)
    # hid_rec = torch.stack(hid_rec)
    return spk_out, hid_rec
plt.imshow(apply(net, ds_test[0][0])[0].detach().T, aspect=5, interpolation="none")


AttributeError: 'RSynaptic' object has no attribute 'mem'

# Run inference

In [10]:
def val_test_loop(dataset, batch_size, net, loss_fn, device, shuffle=True, saved_state_dict=None, label_probabilities=False, regularization=None):
  
  with torch.no_grad():
    if saved_state_dict != None:
        net.load_state_dict(saved_state_dict)
    net.eval()

    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=False)

    batch_loss = []
    batch_acc = []

    for data, labels in loader:
        data = data.to(device).swapaxes(1, 0)
        labels = labels.to(device)

        spk_out, hid_rec = apply(net, data)

        # Validation loss
        if regularization != None:
            # L1 loss on spikes per neuron from the hidden layer
            reg_loss = regularization[0]*torch.mean(torch.sum(hid_rec, 0))
            # L2 loss on total number of spikes from the hidden layer
            reg_loss = reg_loss + regularization[1]*torch.mean(torch.sum(torch.sum(hid_rec, dim=0), dim=1)**2)
            loss_val = loss_fn(spk_out, labels) + reg_loss
        else:
            loss_val = loss_fn(spk_out.sum(0), labels)

        batch_loss.append(loss_val.detach().cpu().item())

        # Accuracy
        act_total_out = torch.sum(spk_out, 0)  # sum over time
        _, neuron_max_act_total_out = torch.max(act_total_out, 1)  # argmax over output units to compare to labels
        batch_acc.extend((neuron_max_act_total_out == labels).detach().cpu().numpy()) # batch_acc.append(np.mean((neuron_max_act_total_out == labels).detach().cpu().numpy()))
    
    if label_probabilities:
        log_softmax_fn = nn.LogSoftmax(dim=-1)
        log_p_y = log_softmax_fn(act_total_out)
        return [np.mean(batch_loss), np.mean(batch_acc)], torch.exp(log_p_y)
    else:
        return [np.mean(batch_loss), np.mean(batch_acc)]


In [11]:
### INFERENCE ON TEST SET

batch_size = 64

input_size = 12 
num_steps = next(iter(ds_test))[0].shape[0]
regularization = [parameters["reg_l1"], parameters["reg_l2"]]


In [12]:
def load_and_val(graph):
    net = #norse.torch.from_nir(graph, reset_method=reset_method)
    return val_test_loop(ds_test, batch_size, net, loss_fn, device, shuffle=False)#, regularization=regularization)    


In [13]:
test_results = load_and_val("braille_subtract.nir")
np.save("snntorch_accuracy_subtract.npy", np.round(test_results[1]*100,2))
print("Test accuracy: {}%".format(np.round(test_results[1]*100,2)))


Test accuracy: 18.57%


In [18]:
test_results = load_and_val("braille_retrained_zero.nir", reset_method=norse.torch.functional.reset_value)
np.save("snntorch_accuracy_zero.npy", np.round(test_results[1]*100,2))
print("Test accuracy: {}%".format(np.round(test_results[1]*100,2)))


Test accuracy: 12.86%
